In [1]:
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
import pandas as pd
from keras import callbacks 
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
# mnist 다운로드 후 DF 변환
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 28*28 이미지를 784차원으로 변경
x_train = x_train.reshape(60000, 784).astype('float32') / 255.0
x_test = x_test.reshape(10000, 784).astype('float32') / 255.0

# target one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
print(f'x_train => {x_train.shape}, {x_train.ndim}차원')
print(f'y_train => {y_train.shape}, {y_train.ndim}차원\n')

x_train => (60000, 784), 2차원
y_train => (60000, 10), 2차원



In [4]:
# 순차적 모델 인스턴스 생성
mModel=Sequential(name='mClassifier')

In [5]:
# 입력층
mModel.add(Dense(16, input_shape=(28*28,), name='In_Layer'))
mModel.add(Activation('relu'))

# 은닉층
mModel.add(Dense(8, activation='relu', name='Hidd1'))
mModel.add(Dense(6, activation='relu', name='Hidd2'))
mModel.add(Dense(4, activation='relu', name='Hidd3'))

# 출력층
mModel.add(Dense(10, activation='softmax', name='Out_Layer'))

In [6]:
mModel.summary()

Model: "mClassifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 In_Layer (Dense)            (None, 16)                12560     
                                                                 
 activation (Activation)     (None, 16)                0         
                                                                 
 Hidd1 (Dense)               (None, 8)                 136       
                                                                 
 Hidd2 (Dense)               (None, 6)                 54        
                                                                 
 Hidd3 (Dense)               (None, 4)                 28        
                                                                 
 Out_Layer (Dense)           (None, 10)                50        
                                                                 
Total params: 12,828
Trainable params: 12,828
Non-train

In [7]:
mModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# 커스텀 콜백 :  에포크가 끝날 때마다 도트(.) 출력해 훈련 진행 과정 표시
class PrintDot(callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0:  print('')
    print('.', end='')

In [9]:
EARLY_STOP = EarlyStopping(monitor='val_loss', patience=10)

In [10]:
filename= './weights.h5'

# ModelCheckpoint 객체 생성
CHECKPOINT = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1, 
                             save_best_only=True, 
                             mode='max', 
                             save_weights_only=True)

In [11]:
# 학습
EPOCHS = 500
PRINT_DOT = PrintDot()

history = mModel.fit(x_train, y_train, 
                        epochs=EPOCHS,
                        validation_split = 0.2, 
                        verbose=0, 
                        callbacks=[EARLY_STOP, PRINT_DOT, CHECKPOINT])
print(f'\n학습 종료')


.
Epoch 1: val_accuracy improved from -inf to 0.88942, saving model to .\weights.h5
.
Epoch 2: val_accuracy improved from 0.88942 to 0.91033, saving model to .\weights.h5
.
Epoch 3: val_accuracy improved from 0.91033 to 0.92125, saving model to .\weights.h5
.
Epoch 4: val_accuracy improved from 0.92125 to 0.92467, saving model to .\weights.h5
.
Epoch 5: val_accuracy improved from 0.92467 to 0.92825, saving model to .\weights.h5
.
Epoch 6: val_accuracy improved from 0.92825 to 0.93125, saving model to .\weights.h5
.
Epoch 7: val_accuracy improved from 0.93125 to 0.93142, saving model to .\weights.h5
.
Epoch 8: val_accuracy improved from 0.93142 to 0.93358, saving model to .\weights.h5
.
Epoch 9: val_accuracy improved from 0.93358 to 0.93517, saving model to .\weights.h5
.
Epoch 10: val_accuracy did not improve from 0.93517
.
Epoch 11: val_accuracy improved from 0.93517 to 0.93675, saving model to .\weights.h5
.
Epoch 12: val_accuracy improved from 0.93675 to 0.93750, saving model to .\

In [12]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
26,0.149662,0.957687,0.252507,0.936833,26
27,0.146440,0.958625,0.240087,0.939833,27
28,0.145594,0.959333,0.245491,0.938000,28
29,0.144128,0.959375,0.242461,0.939250,29
30,0.143485,0.959562,0.248285,0.937083,30


In [13]:
epochs, values = hist['epoch'].values, hist['accuracy'].values

In [14]:
# 커널 충돌이 나서 주석처리함
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 12))

# plt.subplot(2, 1, 1)
# plt.xlabel('Epoch')
# plt.ylabel('[ACCURACY]')
# plt.plot(hist['epoch'], hist['accuracy'], label='Train Error')
# plt.plot(hist['epoch'], hist['val_accuracy'], label='Val Error')
# plt.ylim([0,5])
# plt.title("[CLASSIFICATION ACCURACY]")
# plt.legend()
# plt.show()

: 

In [ ]:
# 검증

loss, accuracy = mModel.evaluate(x_test, y_test, verbose=1)

print(f"\n테스트 세트의 정확도: {accuracy:5.2f}")

313/313 [==============================] - 1s 2ms/step - loss: 0.2256 - accuracy: 0.9448

테스트 세트의 정확도:  0.94
